# Inferentia2 EC2 에 Gradio UI Hugging Face TGI (Text Generation Inference) endpoint 생성
- 이 노트북은 [AWS-Neuron-llama-3-Korean-Bllossom-8B](AWS-Neuron-llama-3-Korean-Bllossom-8B) 모델을 inferentia2 의 TGI Docker 로 서빙하고 있는 포트(8080) 에 대한 Gradio Chat Interface 를 생성 합니다.
    - 참고: [Gradio App](https://www.gradio.app/)

---


## Gradio 설치

In [1]:
! pip install gradio

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
  Using cached huggingface_hub-0.24.5-py3-none-any.whl.metadata (13 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.6/12.6 MB 139.0 MB/s eta 0:00:00
Using cached huggingface_hub-0.24.5-py3-none-any.whl (417 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 195.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.5/19.5 MB 205.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 195.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 143.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 119.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 751.2/751.2 kB 57.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 172.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 134.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 84.7 MB/s eta 

## 사전 과제
- 이 노트북을 실행하는 Inferentia2 EC2의 8080 포트에 에 아래와 같이 TGI Docker 가 실행 중이어야 합니다. 
- ![ready_for_inference.png)](../img/ready_for_inference.png)

## InferenceClient 생성
- http://127.0.0.1:8080 로컬의 8080 포트에 연결하는 클라이언트를 생성 합니다.

In [2]:
import gradio as gr
from huggingface_hub import InferenceClient

client = InferenceClient(model="http://127.0.0.1:8080")

/home/ubuntu/aws-ai-ml-workshop-kr/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 추론 함수 정의

In [3]:
def inference(message, history):
    partial_message = ""
    for token in client.text_generation(message, max_new_tokens=512, stream=True):
        partial_message += token
        yield partial_message

## Chat 인터페이스 생성
- server_name="0.0.0.0", server_port=8081 로 챗 인터페이스 생성
- [AWS-Neuron-llama-3-Korean-Bllossom-8B](AWS-Neuron-llama-3-Korean-Bllossom-8B) 모델을 inferentia2 의 TGI Docker 로 서빙하기

In [4]:
gr.ChatInterface(
    inference,
    chatbot=gr.Chatbot(height=300),
    textbox=gr.Textbox(placeholder="Chat with me!", container=False, scale=7),
    description="This is the demo for Gradio UI consuming TGI endpoint with  AWS-Neuron-llama-3-Korean-Bllossom-8B model, https://huggingface.co/Gonsoo/AWS-Neuron-llama-3-Korean-Bllossom-8B",
    title="Chat with AWS-Neuron-llama-3-Korean-Bllossom-8B model on Inferentia2, inf2.xlarge",
    examples=["1/ 딥러닝에 대해서 알려 주세요. 2/ Are tomatoes vegetables?"],
    retry_btn="Retry",
    undo_btn="Undo",
    clear_btn="Clear",
).queue().launch(server_name="0.0.0.0", server_port=8081, share=False)

Running on local URL:  http://0.0.0.0:8081

To create a public link, set `share=True` in `launch()`.
